In [3]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    print(f"Added {parent_dir} to sys.path")
else:
    print(f"{parent_dir} is already in sys.path")
    
from pipeline.invcov import *

Added /home/mike/corrcal_gpu_pipeline to sys.path
hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.
hello world


# Benchmark Tests for Covariance Inverse Routine

In [2]:
#the main parametes describing our problem.
n_bl = 10000
n_eig = 3
n_src = 6
xp = cp  #run things on the gpu using cupy

#random array of edges for the diffuse matrix
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 10))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
sim_noise_mat = xp.random.rand(n_bl)**2   #in principle this is squared since is a variance
sim_diff_mat = xp.random.rand(n_bl, n_eig)
sim_src_mat = xp.random.rand(n_bl, n_src)

#perform cupy benchmark
test_results = str(benchmark(inverse_covariance, (sim_noise_mat, sim_diff_mat, sim_src_mat, edges, xp), n_repeat=100))
test_results = test_results.split()
cpu_t = float(test_results[3])/1e6
gpu_t = float(test_results[14])/1e6
print(f"Time on cpu: {cpu_t:.6f}s")
print(f"Time on gpu: {gpu_t:.6f}s")


The edges of the redundant blocks have indices[    0   257  1193  1689  2335  2888  6634  8826  9322  9371  9566 10000]
Time on cpu: 0.020478s
Time on gpu: 0.020503s


cpu = 0.0086

gpu = 0.0091

cpu = 0.026

gpu = 0.026

Seems to run ***way*** faster using numpy and thus on the cpu. Need to figure out why this is...

## Below is veery rough work trying to understand why the time differs in the way it does between the cpu and the gpu ... 

Update: Turns out to be the zeropad function which we will work on elsewhere 

In [1]:
import time

# Function to time the execution
def time_function(func, *args):
    start_time = time.time()
    func(*args)
    return time.time() - start_time

# Timing CPU execution
xp = np
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 10))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
sim_noise_mat = xp.random.rand(n_bl)**2
sim_diff_mat = xp.random.rand(n_bl, n_eig)
sim_src_mat = xp.random.rand(n_bl, n_src)
cpu_time = time_function(inverse_covariance, sim_noise_mat, sim_diff_mat, sim_src_mat, edges, xp)
print(f"CPU execution time: {cpu_time:.6f}s")

# Timing GPU execution
xp = cp
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 10))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
sim_noise_mat = xp.random.rand(n_bl)**2
sim_diff_mat = xp.random.rand(n_bl, n_eig)
sim_src_mat = xp.random.rand(n_bl, n_src)
gpu_time = time_function(inverse_covariance, sim_noise_mat, sim_diff_mat, sim_src_mat, edges, xp)
print(f"GPU execution time: {gpu_time:.6f}s")


NameError: name 'np' is not defined

In [4]:
import cupy as cp
import time

def transfer_and_time(xp, *arrays):
    start = time.time()
    arrays = [xp.asarray(arr) for arr in arrays]
    transfer_time = time.time() - start
    return transfer_time, arrays

# Measure transfer time
edges = np.unique(np.random.randint(1, n_bl-1, size = 10))
edges = np.concatenate((np.array([0]), edges, np.array([n_bl])))
transfer_time, (sim_noise_mat, sim_diff_mat, sim_src_mat) = transfer_and_time(cp, 
    np.random.rand(n_bl)**2, 
    np.random.rand(n_bl, n_eig), 
    np.random.rand(n_bl, n_src)
)
print(f"Data transfer time: {transfer_time:.6f}s")


Data transfer time: 0.001188s


In [6]:
from cupyx.profiler import benchmark

def benchmark_inverse_covariance():
    result = str(benchmark(inverse_covariance, (sim_noise_mat, sim_diff_mat, sim_src_mat, edges, cp), n_repeat=10))
    result = result.split()
    cpu_t = float(result[3]) / 1e6
    gpu_t = float(result[14]) / 1e6
    print(f"Time on CPU: {cpu_t:.6f}s")
    print(f"Time on GPU: {gpu_t:.6f}s")

benchmark_inverse_covariance()


Time on CPU: 0.008077s
Time on GPU: 0.008838s


In [2]:
import cupy as cp
import numpy as np
from cupyx.profiler import benchmark

def matmul(A, B, xp):
    result = xp.zeros((A.shape[0], B.shape[1]))
    for row in range(A.shape[0]):
        for col in range(B.shape[1]):
            for k in range(B.shape[0]):
                result[row, col] += A[row, k]*B[k, col] 
    return result

In [7]:
xp = np
A = xp.random.rand(10, 10)
B = xp.random.rand(10, 10)

In [8]:

result = str(benchmark(matmul, (A, B, xp), n_repeat=100))
result = result.split()
cpu_t = float(result[3]) / 1e6
gpu_t = float(result[14]) / 1e6
print(f"Time on CPU: {cpu_t:.6f}s")
print(f"Time on GPU: {gpu_t:.6f}s")


Time on CPU: 0.000645s
Time on GPU: 0.001094s


In [33]:
#let's benchmark the zeropad functions

#the main parametes describing our problem.
n_bl = 120000
n_eig = 10
n_src = 5
xp = np  #run things on the gpu using cupy

#random array of edges for the diffuse matrix
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 500))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
# print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
sim_noise_mat = xp.random.rand(n_bl)**2   #in principle this is squared since is a variance
sim_diff_mat = xp.random.rand(n_bl, n_eig)
sim_src_mat = xp.random.rand(n_bl, n_src)


result = str(benchmark(zeropad, (sim_diff_mat, edges, xp), n_repeat=1000))
result = result.split()
cpu_t = float(result[3]) / 1e6
gpu_t = float(result[14]) / 1e6
print(f"Time on CPU: {cpu_t:.6f}s")
print(f"Time on GPU: {gpu_t:.6f}s")

Time on CPU: 0.011497s
Time on GPU: 0.012087s


In [65]:
import cupy as cp

# Define the CUDA kernel for zero-padding
zeropad_kernel = cp.RawKernel(r'''
extern "C" __global__
void zeropad(const int* edges, const float* array, float* out, int n_blocks, int largest_block, int array_width) {
    int block_idx = blockIdx.x;
    int thread_idx = threadIdx.x;
    int start = edges[block_idx];
    int stop = edges[block_idx + 1];
    int offset = block_idx * largest_block;

    if (thread_idx < (stop - start)) {
        out[offset + thread_idx] = array[start + thread_idx];
    }
}
''', 'zeropad')

def zeropad_cuda(array, edges, xp):
    edges = xp.asarray(edges, dtype=cp.int32)
    array = xp.asarray(array, dtype=cp.float32)
    
    largest_block = xp.diff(edges).max()
    n_blocks = edges.size - 1

    if array.ndim == 1:  # should only be the case for the noise matrix
        out = xp.zeros((n_blocks, int(largest_block)), dtype=cp.float32)
    else:
        out = xp.zeros((n_blocks, int(largest_block), int(array.shape[1])), dtype=cp.float32)

    # Launch the kernel
    threads_per_block = 256
    blocks_per_grid = n_blocks

    if array.ndim == 1:
        zeropad_kernel((blocks_per_grid,), (threads_per_block,), 
                       (edges, array, out, n_blocks, int(largest_block), 0))
    else:
        for i in range(array.shape[1]):
            zeropad_kernel((blocks_per_grid,), (threads_per_block,), 
                           (edges, array[:, i], out[:, :, i], n_blocks, int(largest_block), array.shape[1]))

    return out


In [4]:
#the main parametes describing our problem.
n_bl = 10
n_eig = 3
n_src = 5
xp = cp  #run things on the gpu using cupy

#random array of edges for the diffuse matrix
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 10))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
sim_noise_mat = xp.random.rand(n_bl)**2   #in principle this is squared since is a variance
sim_diff_mat = xp.random.rand(n_bl, n_eig)
sim_src_mat = xp.random.rand(n_bl, n_src)


result = str(benchmark(zeropad_cuda, (sim_diff_mat, edges, xp), n_repeat=10000))
result = result.split()
cpu_t = float(result[3]) / 1e6
gpu_t = float(result[14]) / 1e6
print(f"Time on CPU: {cpu_t:.6f}s")
print(f"Time on GPU: {gpu_t:.6f}s")

The edges of the redundant blocks have indices[ 0  1  2  4  5  7  8 10]


NameError: name 'zeropad_cuda' is not defined

In [118]:
print(zeropad_cuda(sim_diff_mat, edges, cp))

[[[0.31668377 0.9918387  0.05347418]
  [0.05347418 0.6461567  0.6461567 ]]

 [[0.5948217  0.5948217  0.20444898]
  [0.20444898 0.28285316 0.28285316]]

 [[0.56092954 0.56092954 0.84872013]
  [0.581425   0.9728917  0.7077299 ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]]]
